In [45]:
import sys
import psql_handler
from image_tags import get_tags_from_cache
from psql_handler import create_db_connection
import os 
from dotenv import load_dotenv
from multiprocessing import Pool
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
load_dotenv()
cache_dir = os.getenv("CACHE_DIR")

In [46]:
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
engine = create_engine('postgresql+pg8000://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+':5432/'+DB_NAME)

In [47]:
def get_all_tags(engine):
    QUERRY = "SELECT * FROM public.oc_systemtag"
    try:
        df_tags = pd.read_sql_query(sqlalchemy.text(QUERRY), engine)
        print("got tags")
    except Exception as e:
        print("Error:", e)
    return df_tags

In [48]:
tags = pd.read_csv('image_cache/tags.csv')
df_file_loc = pd.read_csv( "image_cache/db.csv")


# Map tags to fileids
df_all_tag = get_all_tags(engine)
df_map = df_file_loc[['fileid', 'tags']]
df_map = df_map.assign(tags=df_map['tags'].str.split(';'))
df_map = df_map.explode('tags')
df_map['tags'] = df_map['tags'].str.strip()




# Assuming:
# df_all_tags has columns: id, name
# df_source has columns: fileid, tags





got tags


In [49]:
# Merge df_source with df_all_tags on tag name
df_result = pd.merge(
    df_map,
    df_all_tag,
    left_on='tags',
    right_on='name',
    how='inner'
)

# Select only the columns needed
df_result = df_result[['fileid', 'id']]
df_result.columns = ['objectid', 'systemtagid']
df_result['objecttype'] = "files"

In [50]:
df_result

,objectid,systemtagid,objecttype
0,19790,3,files
1,19790,4,files
2,19790,5,files
3,19790,6,files
4,19790,7,files
...,...,...,...
276,19552,91,files
277,19552,14,files
278,19552,6,files
279,19552,16,files


In [53]:
df_result.to_sql('oc_systemtag_object_mapping', engine, if_exists="append", index=False)


281